# Panaroo Preparasi

In [1]:
pwd()

'/home/hnasrulloh/Works/pangenomes/notebooks'

In [2]:
from os import path
from collections import defaultdict

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

In [3]:
DATA_DIR = path.join(path.pardir, 'data')

SAMPLES_DIR = path.join(DATA_DIR, 'samples')
QC_DIR = path.join(DATA_DIR, 'qc')

## 1. Deteksi Outlier

In [4]:
NGENES_TXT = path.join(QC_DIR, 'ngenes.txt')

genes = pd.read_csv(NGENES_TXT, sep='\t', dtype={'sample': str, 'no_genes': np.float64})
genes.columns = ['genome_id', 'total_genes']
genes

,genome_id,total_genes
0,1267355.3,4365.0
1,1267356.3,4345.0
2,1267358.3,4358.0
3,1267359.3,4331.0
4,1267364.3,4355.0
...,...,...
707,1773.203,4312.0
708,1773.204,4269.0
709,652616.4,4372.0
710,83331.31,4346.0


### 1.1 Outliers Total Gen

In [5]:
upper_outlier = genes[genes['genome_id'] == '1400918.3'].total_genes.to_list()[0]
lower_outlier = genes[genes['genome_id'] == '1773.200'].total_genes.to_list()[0]

genes = genes[(genes.total_genes <= upper_outlier) & (genes.total_genes >= lower_outlier)]
genes

,genome_id,total_genes
0,1267355.3,4365.0
1,1267356.3,4345.0
2,1267358.3,4358.0
3,1267359.3,4331.0
4,1267364.3,4355.0
...,...,...
706,1773.202,4307.0
707,1773.203,4312.0
709,652616.4,4372.0
710,83331.31,4346.0


### 1.2 Outlier dengan MDS

In [6]:
mds_outliers = ['1324246.3', '1295760.3']

genes = genes[~genes.genome_id.isin(mds_outliers)]
genes

,genome_id,total_genes
0,1267355.3,4365.0
1,1267356.3,4345.0
2,1267358.3,4358.0
3,1267359.3,4331.0
4,1267364.3,4355.0
...,...,...
706,1773.202,4307.0
707,1773.203,4312.0
709,652616.4,4372.0
710,83331.31,4346.0


## 2. Export

### 2.1 Excel: AMR dan Genomes

In [7]:
AMR_BEFORE_MASH = path.join(DATA_DIR, 'amr_before_mash.xlsx')
df_amr = pd.read_excel(AMR_BEFORE_MASH, dtype=str)

df_amr = df_amr[df_amr.genome_id.isin(genes.genome_id)]

AMR_READY = path.join(DATA_DIR, 'amr_ready.xlsx')
df_amr.to_excel(AMR_READY, index=False)

df_amr

,genome_id,genome_name,antibiotic,phenotype,measurement,measurement_unit,laboratory_typing_method
0,1438833.3,Mycobacterium tuberculosis 1010SM,rifampin,R,40,mg/L,Agar proportion
1,1438835.3,Mycobacterium tuberculosis 1173CS,ethambutol,S,2,mg/L,Agar proportion
2,1438835.3,Mycobacterium tuberculosis 1173CS,rifampin,R,40,mg/L,Agar proportion
3,1438837.3,Mycobacterium tuberculosis 1429BH,ethambutol,S,2,mg/L,Agar proportion
4,1438837.3,Mycobacterium tuberculosis 1429BH,rifampin,R,40,mg/L,Agar proportion
...,...,...,...,...,...,...,...
1294,1773.200,Mycobacterium tuberculosis strain MAL010077,isoniazid,R,1,mg/L,Agar dilution
1298,1773.202,Mycobacterium tuberculosis strain MAL020139,isoniazid,R,1,mg/L,Agar dilution
1299,1773.203,Mycobacterium tuberculosis strain MAL020151,ethambutol,S,0.1,mg/L,Broth dilution
1300,1773.203,Mycobacterium tuberculosis strain MAL020151,isoniazid,S,0.1,mg/L,Broth dilution


In [8]:
GENOMES_BEFORE_MASH = path.join(DATA_DIR, 'genomes_before_mash.xlsx')
df_genomes = pd.read_excel(GENOMES_BEFORE_MASH, dtype=str)

df_genomes = df_genomes[df_genomes.genome_id.isin(genes.genome_id)]

GENOMES_READY = path.join(DATA_DIR, 'genomes_ready.xlsx')
df_genomes.to_excel(GENOMES_READY, index=False)

df_genomes

,genome_id,genome_name,strain,taxon_id,genome_length,gc_content,fine_consistency,checkm_completeness,checkm_contamination,completion_date,...,sequences,patric_cds,refseq_cds,isolation_site,isolation_source,isolation_comments,collection_date,geographic_location,host_name,comments
0,1438833.3,Mycobacterium tuberculosis 1010SM,1010SM,1438833,4406079,65.6,99.3,100,0,2014-04-29T00:00:00Z,...,NaN,4372,4006,NaN,neg,NaN,2013,Romania,"Human, Homo sapiens",TB-ARC - Romania
1,1438835.3,Mycobacterium tuberculosis 1173CS,1173CS,1438835,4383014,65.6,99.2,100,0,2014-04-29T00:00:00Z,...,NaN,4333,3982,NaN,Bodily fluid,NaN,2012,Romania,"Human, Homo sapiens",TB-ARC - Romania
2,1438837.3,Mycobacterium tuberculosis 1429BH,1429BH,1438837,4417210,65.6,99.3,100,0,2014-04-29T00:00:00Z,...,NaN,4384,4022,NaN,Bodily fluid,NaN,2013,Romania,"Human, Homo sapiens",TB-ARC - Romania
3,1438838.3,Mycobacterium tuberculosis 1430BH,1430BH,1438838,4418670,65.6,99.3,100,0,2014-04-29T00:00:00Z,...,NaN,4383,4023,NaN,Bodily fluid,NaN,2013,Romania,"Human, Homo sapiens",TB-ARC - Romania
4,1438842.3,Mycobacterium tuberculosis 1755BN,1755BN,1438842,4394400,65.6,99.3,100,0,2014-04-29T00:00:00Z,...,NaN,4360,3987,NaN,Bodily fluid,NaN,2013,Romania,"Human, Homo sapiens",TB-ARC - Romania
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704,652616.4,Mycobacterium tuberculosis str. Erdman = ATCC ...,Erdman,652616,4417384,65.65,99.3,100,0,NaN,...,NaN,4372,4006,NaN,NaN,NaN,NaN,NaN,NaN,NaN
705,1773.199,Mycobacterium tuberculosis strain MAL010075,MAL010075,1773,4421633,65.6,99.3,100,0,2014-04-29T00:00:00Z,...,NaN,4320,4006,NaN,NaN,NaN,2007,Mali,"Human, Homo sapiens",TB-ARC - Mali
706,1773.200,Mycobacterium tuberculosis strain MAL010077,MAL010077,1773,4421308,65.6,99.3,100,0,2014-04-29T00:00:00Z,...,NaN,4304,4006,NaN,NaN,NaN,NaN,Mali,"Human, Homo sapiens",TB-ARC - Mali
708,1773.202,Mycobacterium tuberculosis strain MAL020139,MAL020139,1773,4413724,65.6,99.4,100,0,2014-04-29T00:00:00Z,...,NaN,4307,4003,NaN,NaN,NaN,2007,Mali,"Human, Homo sapiens",TB-ARC - Mali


### 2.2 Panaroo

**INGAT** tambahkan genome reference lagi!

- Genome ID: **83332.12**
- Genome Name: **Mycobacterium tuberculosis H37Rv**

In [9]:
reference_id = '83332.12'

In [10]:
ID_LIST_TXT = path.join(SAMPLES_DIR, '_id_ready.txt')
with open(ID_LIST_TXT, 'w') as f:
    gids = [reference_id]
    gids.extend(df_amr.genome_id.unique())
    for gid in gids:
        f.write(f'{gid}\n')

selected_antibiotics = df_amr.antibiotic.unique()
for antibiotic in selected_antibiotics:    
    PANAROO_LIST_TXT = path.join(SAMPLES_DIR, f'_panaroo_{antibiotic}_ready.txt')
    gids = [reference_id]
    gids.extend(df_amr[df_amr.antibiotic == antibiotic]['genome_id'].to_list())
    with open(PANAROO_LIST_TXT, 'w') as f:
        for gid in gids:
            f.write(f'{gid}.PATRIC.gff {gid}.fna\n')

In [11]:
# Plot the distributions
dist = defaultdict(int)
for d in df_amr[['antibiotic', 'phenotype']].itertuples():
    name = f'{d.antibiotic}:{d.phenotype}'
    dist[name] += 1
    
antibiotics = set([a.split(':')[0] for a in dist.keys()])
counts = [(dist[f'{a}:R'], dist[f'{a}:S']) for a in antibiotics]
pd.DataFrame(counts, index=list(antibiotics), columns=['resistant', 'susceptible'])

,resistant,susceptible
rifampin,179,201
ethambutol,160,89
isoniazid,406,175
